In [198]:
import overpy
from geopy.geocoders import Photon
from geopy.distance import geodesic
import pandas as pd

In [199]:
def geocode_address(address:str):
    '''Receives an address with street name, number, zip code, city and country
    and returns latitude and longitude'''
    geolocator = Photon(user_agent="measurements")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude


In [200]:
def calculate_distance(coord1, coord2):
    '''Receives two coordinates and returns the distance between them in meters'''
    return geodesic(coord1, coord2).meters

In [203]:
def find_nearby_places(address:str, place_type:str, radius:int=500):
    '''Function to find places of a specific type near a given address and radius in meters.
    Available place types are: "restaurant", "bar", "gym", "park", "cafe", "hospital", "school"'''
    latitude, longitude = geocode_address(address)
    origin = (latitude, longitude)

    # Initialize the Overpass API
    api = overpy.Overpass()

    # Define place type mapping to appropriate OSM tags
    place_type_mapping = {
        'restaurant': '[amenity=restaurant]',
        'bar': '[amenity=pub]',
        'gym': '[leisure=fitness_centre]',
        'park': '[leisure=park]',
        'cafe': '[amenity=cafe]',
        'hospital': '[amenity=hospital]',
        'school': '[amenity=school]'
    }

    # Get the appropriate OSM tag for the given place type
    osm_tag = place_type_mapping.get(place_type.lower())

    if not osm_tag:
        raise ValueError(f"Unsupported place type: {place_type}")

     # Formulate the Overpass API query
    query = f"""
    [out:json];
    (
        node{osm_tag}(around:{radius},{latitude},{longitude});
        way{osm_tag}(around:{radius},{latitude},{longitude});
        relation{osm_tag}(around:{radius},{latitude},{longitude});
    );
    out center;
    """

    # Query the Overpass API
    result = api.query(query)

    # Store results in a list
    places = []

    for node in result.nodes:
        node_coords = (node.lat, node.lon)
        distance = calculate_distance(origin, node_coords)
        places.append({
            'name': node.tags.get('name', 'N/A'),
            'latitude': node.lat,
            'longitude': node.lon,
            'distance': distance
        })

    for way in result.ways:
        if way.center_lat is not None and way.center_lon is not None:
            way_coords = (way.center_lat, way.center_lon)
            distance = calculate_distance(origin, way_coords)
            places.append({
                'name': way.tags.get('name', 'N/A'),
                'latitude': way.center_lat,
                'longitude': way.center_lon,
                'distance': distance
            })

    for relation in result.relations:
        if relation.center_lat is not None and relation.center_lon is not None:
            relation_coords = (relation.center_lat, relation.center_lon)
            distance = calculate_distance(origin, relation_coords)
            places.append({
                'name': relation.tags.get('name', 'N/A'),
                'latitude': relation.center_lat,
                'longitude': relation.center_lon,
                'distance': distance
            })

    # Sort the list by distance
    places_sorted = sorted(places, key=lambda x: x['distance'])

    return pd.DataFrame(places_sorted)

In [205]:
find_nearby_places("Rudi-Dutschke-Straße 26, 10969 Berlin, Germany", 'restaurant', 500)

,name,description,latitude,longitude,distance
0,Tim Raue,N/A,52.5068830,13.3913836,0.000000
1,Ristorante Sale e Tabacchi,N/A,52.5066487,13.3913866,26.072988
2,Babami,N/A,52.5076191,13.3916108,83.351007
3,Barcelona,N/A,52.5063040,13.3903978,92.906167
4,Hatay Ocakbaşi,N/A,52.5067848,13.3896887,115.600833
5,Sapori del Sud,N/A,52.5080178,13.3919864,132.744573
6,Caramel,N/A,52.5076992,13.3930233,143.681258
7,Ishin,N/A,52.5079532,13.3926131,145.434771
8,Viet Bowl,N/A,52.5079443,13.3929391,158.436306
9,Shima,N/A,52.5054590,13.3905386,168.526186


In [171]:
address = "Rudi-Dutschke-Straße 26, 10969 Berlin, Germany"
latitude, longitude = geocode_address(address)
origin = (latitude, longitude)

In [122]:
api = overpy.Overpass()

In [153]:
query = f"""
[out:json];
node
  [leisure=fitness_centre]
  (around:2000,{latitude},{longitude});
out body;
"""

In [154]:
result = api.query(query)

In [155]:
places = []

for node in result.nodes:
    node_coords = (node.lat, node.lon)
    distance = calculate_distance(origin, node_coords)
    places.append({
        'name': node.tags.get('name', 'N/A'),
        'latitude': node.lat,
        'longitude': node.lon,
        'distance': distance
    })

# Sort the list by distance
places_sorted = sorted(places, key=lambda x: x['distance'])

# Print the sorted results
for place in places_sorted:
    print(f"Place: {place['name']}, Lat: {place['latitude']}, Lon: {place['longitude']}, Distance: {place['distance']:.6f} meters")

Place: Bodystreet, Lat: 52.5079287, Lon: 13.3917379, Distance: 118.823019 meters
Place: McFit, Lat: 52.5106479, Lon: 13.3974279, Distance: 586.459598 meters
Place: Holmes Place, Lat: 52.5123998, Lon: 13.3908216, Distance: 615.077747 meters
Place: Forde Fitness, Lat: 52.5025396, Lon: 13.3844025, Distance: 676.985897 meters
Place: AllStar Gym, Lat: 52.5101818, Lon: 13.3814085, Distance: 770.361361 meters
Place: Beat81, Lat: 52.5110212, Lon: 13.3818076, Distance: 796.728956 meters
Place: Fitness First, Lat: 52.5139697, Lon: 13.3954862, Distance: 836.334461 meters
Place: Holmes Place, Lat: 52.5054645, Lon: 13.3752868, Distance: 1104.323405 meters
Place: X-Step, Lat: 52.5006590, Lon: 13.3756597, Distance: 1272.677648 meters
Place: DocOrtho, Lat: 52.5191433, Lon: 13.3891020, Distance: 1373.055602 meters
Place: SunYoga, Lat: 52.4934118, Lon: 13.3886695, Distance: 1510.321263 meters
Place: PRO-STAGE, Lat: 52.4939117, Lon: 13.3982992, Distance: 1517.885556 meters
Place: Ladycompany - Fitness fü